In [18]:
using JSON

# Specify the file name and location
filename = "tiny.json"

# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = true  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]

for vehicle in vehicles
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: regular")
    else
        # Otherwise, print the actual vehicle type
        println("Vehicle ID: ", vehicle["id"], ", Type: ", vehicle["type"])
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end



Shops:
Any[Dict{String, Any}("name" => "body", "resequencing_lag" => 1), Dict{String, Any}("name" => "paint", "resequencing_lag" => 2), Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)]
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Constraint: Dict{String, Any}("max_vehicles" => 4, "cost" => 2, "min_vehicles" => 2, "id" => 1, "vehicles" => Any[1, 2, 4], "type" => "batch_size", "shop" => "body")
Constraint: Dict{String, Any}("cost" => 3, "partition" => Any[Any[1, 2], Any[3, 4, 5]], "id" => 2, "type" => "lot_change", "shop" => "paint")
Constraint: Dict{String, Any}("max_vehicles" => 2, "cost" => 2, "id" => 3, "window_size" => 3, "vehicles" => Any[1, 2, 5], "type" => "rolling_window", "shop" => "assembly")
Constraint: Dict{String, Any}("max_vehicles" => 3, "cost" => 3, "min_vehicles" => 2, "id" => 4, "vehicles" => Any[1, 2, 3], "type" => "batch_size", "shop" => "assembly")
Constraint: Dict{String, Any}("max_ve

In [42]:
resequencing_cost = parameters["resequencing_cost"]
S = length(shops)    
println("Number of shops: ", number_of_shops)
n = length(vehicles)
println("number_of_vehicles, ",number_of_vehicles)

Number of shops: 3
number_of_vehicles, 5


Here is an example of how an MILP solver works in Julia:

**Problem:**

Maximize 

$$
60x_1 + 100x_2 + 120x_3
$$

subject to 

$$
10x_1 + 20x_2 + 30x_3 \leq 50
$$

$$
x_1, x_2, x_3 \in \{0, 1\}
$$


In [36]:
using JuMP
using GLPK

model = Model(GLPK.Optimizer)

# Define variables
@variable(model, x[1:3]>=0, Int)

# Add constraints
@constraint(model, 10x[1] + 20x[2] + 30x[3] <= 50)

# Set the objective
@objective(model, Max, 60x[1] + 100x[2] + 120x[3])

# Solve the problem
optimize!(model)

# Display results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution:")
    println("x = ", value.(x))
    println("Objective value = ", objective_value(model))
else
    println("No optimal solution found.")
end

Optimal solution:
x = [5.0, 0.0, 0.0]
Objective value = 300.0


In [43]:
using JuMP
using GLPK

# Example cost coefficients (adjust as needed)
c_s = rand(S)  # Cost for b
c_l = rand(S)  # Cost for f
c_r = rand(S)  # Cost for z
c_b = rand(S)  # Cost for g

# Create the JuMP model
model = Model(GLPK.Optimizer)

# Define binary and integer variables
@variable(model, x[1:S, 1:n, 1:n], Bin)
@variable(model, b[1:S, 1:n], Bin)
@variable(model, f[1:S, 1:n-1], Bin)
@variable(model, z[1:S, 1:n-w_r+1] >= 0, Int)
@variable(model, p[1:S, 1:n], Bin)
@variable(model, g[1:S, 1:n, 1:n], Bin)

# Define continuous variable y
@variable(model, y[1:S, 1:n] >= 0)

# Objective function
@objective(model, Min,
    sum(c_s[s] * b[s, v] for s = 1:S, v = 1:n) +
    sum(c_l[s] * f[s, v, t] for s = 1:S, v = 1:n-1, t = 1:n-1) +
    X * sum(c_r[s] * z[s, r, t] for s = 1:S, r = 1:n-w_r+1, t = 1:n-w_r+1) +
    sum(c_b[s] * g[s, v, t, t_prime] * (t_prime - t + 1) for s = 1:S, v = 1:n, t = 1:n-1, t_prime = t:n)
)

# Constraints
@constraint(model, [s=1:S, v=1:n], sum(x[s, v, t] for t=1:n) == 1)  # sum over t
@constraint(model, [s=1:S, t=1:n], sum(x[s, v, t] for v=1:n) == 1)  # sum over v

@constraint(model, [s=1:S, v=1:n], y[s, v] >= sum(t * x[s, v, t] for t=1:n) - M)  # y constraints

@constraint(model, [s=1:S, v=1:n], b[s, v] >= y[s, v] / M)

@constraint(model, [s=1:S, l=1:n-1, t=1:n-1], f[s, l, t] >= sum(x[s, t, v] for v = 1:n) - sum(x[s, t+1, v] for v = 1:n))  # Lot consistency constraints

@constraint(model, [s=1:S, r=1:n-w_r+1, t=1:n-w_r+1], z[s, r, t] >= sum(x[s, r, t'] for t' = t:t+w_r-1) - M)  # Lot consistency

@constraint(model, [s=1:S, t=1:n], p[s, t] >= x[s, t, v])  # New variable p

# Further constraints for n, m, l, g
@constraint(model, [s=1:S, b=1:n, t=2:n], n[s, b, t] >= (1 - p[s, b]))
@constraint(model, [s=1:S, b=1:n, t=1:n], m[s, b, t] >= p[s, b])
@constraint(model, [s=1:S, b=1:n, t=1:n], l[s, b, t] >= (1 - p[s, b]))

# Solve the model
optimize!(model)

# Retrieve and print the optimal value
println("Optimal value: ", objective_value(model))

# Retrieve the values of variables (e.g., x, b, f, etc.)
println("x values: ", value.(x))
println("b values: ", value.(b))
println("f values: ", value.(f))
println("z values: ", value.(z))
println("g values: ", value.(g))


BoundsError: BoundsError: attempt to access 3×4 Matrix{VariableRef} at index [1, 1, 2]

In [ ]:
using JuMP
using GLPK # Or use another solver like Gurobi or CBC
#this is a basic example, if will fix later

# Define the optimization model
model = Model(GLPK.Optimizer)
R =  4 
L_s = [1 for i in 1:S]
K_s = [1 for i i in 1:S]
B_s = [1 for i in 1:S]
# 1. Binary decision variables for xsvt (Vehicle assignment at position t in sequence s)
@variable(model, x[1:S, 1:n, 1:n], Bin)  # S = number of sequences, V = number of vehicles, T = number of positions

# 2. Continuous decision variables for ysv (Qsuantity associated with sequence s and vehicle v)
@variable(model, y[1:S, 1:n] >= 0)  # ysv >= 0

# 3. Binary decision variables for bsv (Boundary condition on y)
@variable(model, b[1:S, 1:n], Bin)  # bsv ∈ {0, 1}
# 4. Continuous decision variables for u(s,t,ℓ,k) (Lot assignment variables under condition ℓ for sequence s at position t)
@variable(model, u[1:S, 1:n, L_s[1:S], 1:K] >= 0)  # L = number of conditions, K = number of lots

# 5. Binary decision variables for f(s,ℓ,t) (Lot change condition)
@variable(model, f[1:S, , 1:n], Bin)  # f(s,ℓ,t) ∈ {0, 1}

# 6. Continuous decision variables for z(s,r,t) (Consistency of lot usage over window r for sequence s)
@variable(model, z[1:S, 1:R, 1:n] >= 0)  # R = number of windows

# 7. Binary decision variables for p(s,t,b) (Block assignment at position t in sequence s)
@variable(model, p[1:S, 1:n, 1:B], Bin)  # B = number of blocks

# 8. Binary decision variables for g(s,b,t,t') (Block relation between positions t and t' for sequence s and block b)
@variable(model, g[1:S, 1:B, 1:n, 1:n], Bin)  # g(s,b,t,t') ∈ {0, 1}


3×8×5×5 Array{VariableRef, 4}:
[:, :, 1, 1] =
 g[1,1,1,1]  g[1,2,1,1]  g[1,3,1,1]  …  g[1,6,1,1]  g[1,7,1,1]  g[1,8,1,1]
 g[2,1,1,1]  g[2,2,1,1]  g[2,3,1,1]     g[2,6,1,1]  g[2,7,1,1]  g[2,8,1,1]
 g[3,1,1,1]  g[3,2,1,1]  g[3,3,1,1]     g[3,6,1,1]  g[3,7,1,1]  g[3,8,1,1]

[:, :, 2, 1] =
 g[1,1,2,1]  g[1,2,2,1]  g[1,3,2,1]  …  g[1,6,2,1]  g[1,7,2,1]  g[1,8,2,1]
 g[2,1,2,1]  g[2,2,2,1]  g[2,3,2,1]     g[2,6,2,1]  g[2,7,2,1]  g[2,8,2,1]
 g[3,1,2,1]  g[3,2,2,1]  g[3,3,2,1]     g[3,6,2,1]  g[3,7,2,1]  g[3,8,2,1]

[:, :, 3, 1] =
 g[1,1,3,1]  g[1,2,3,1]  g[1,3,3,1]  …  g[1,6,3,1]  g[1,7,3,1]  g[1,8,3,1]
 g[2,1,3,1]  g[2,2,3,1]  g[2,3,3,1]     g[2,6,3,1]  g[2,7,3,1]  g[2,8,3,1]
 g[3,1,3,1]  g[3,2,3,1]  g[3,3,3,1]     g[3,6,3,1]  g[3,7,3,1]  g[3,8,3,1]

[:, :, 4, 1] =
 g[1,1,4,1]  g[1,2,4,1]  g[1,3,4,1]  …  g[1,6,4,1]  g[1,7,4,1]  g[1,8,4,1]
 g[2,1,4,1]  g[2,2,4,1]  g[2,3,4,1]     g[2,6,4,1]  g[2,7,4,1]  g[2,8,4,1]
 g[3,1,4,1]  g[3,2,4,1]  g[3,3,4,1]     g[3,6,4,1]  g[3,7,4,1]  g[3,8,4,1]

[:, :

In [ ]:
@objective(model, Min,
    sum(c_x * x[s, t, v] for s in 1:S, t in 1:n, v in 1:n) +
    sum(c_y * y[s, v] for s in 1:S, v in 1:n) +
    sum(c_b * b[s, v] for s in 1:S, v in 1:n) +
    sum(c_u * u[s, t, l, k] for s in 1:S, t in 1:n, l in 1:L_s[s], k in 1:K) +
    sum(c_f * f[s, l, t] for s in 1:S, l in 1:L_s[s], t in 1:n) +
    sum(c_z * z[s, r, t] for s in 1:S, r in 1:R, t in 1:n) +
    sum(c_p * p[s, t, b] for s in 1:S, t in 1:n, b in 1:B) +
    sum(c_g * g[s, b, t, t_prime] for s in 1:S, b in 1:B, t in 1:n, t_prime in 1:n)
)

In [ ]:
# 1. Vehicle assignment constraint: each vehicle is assigned to exactly one position in each sequence
@constraint(model, [s in 1:S, t in 1:n], sum(x[s, t, v] for v in 1:n) == 1)

# 2. Boundary condition constraint: y_sv ≥ sum(t * x_sv_t) - Δ_s - sum(t * x_(s+1)_v_t)
@constraint(model, [s in 1:S, v in 1:n], y[s, v] >= sum(t * x[s, t, v] for t in 1:n) - Δ_s - sum(t * x[s+1, t, v] for t in 1:n))

# 3. Boundary condition binary constraint: b_sv ≥ y_sv / M
@constraint(model, [s in 1:S, v in 1:n], b[s, v] >= y[s, v] / M)

# 4. Lot assignment constraint: u_sv_l_k ≥ 0 for all s, t, l, k
@constraint(model, [s in 1:S, t in 1:n, l in 1:L_s[s], k in 1:K], u[s, t, l, k] >= 0)

# 5. Lot change condition binary constraint: f_sl_k ∈ {0, 1}
@constraint(model, [s in 1:S, l in 1:L_s[s], t in 1:n], f[s, l, t] in [0, 1])

# 6. Consistency of lot usage constraint: z_srt ≥ 0
@constraint(model, [s in 1:S, r in 1:R, t in 1:n], z[s, r, t] >= 0)

# 7. Block assignment binary constraint: p_st_b ∈ {0, 1}
@constraint(model, [s in 1:S, t in 1:n, b in 1:B], p[s, t, b] in [0, 1])

# 8. Block relation binary constraint: g_sb_tt' ∈ {0, 1}
@constraint(model, [s in 1:S, b in 1:B, t in 1:n, t_prime in 1:n], g[s, b, t, t_prime] in [0, 1])
